In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import sklearn as skl
import random as rdm
import pickle as pkl
from sklearn.model_selection import train_test_split
from keras.applications.densenet import DenseNet169
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from datetime import datetime

In [2]:
data = pd.read_csv("/kaggle/input/bitmoji-faces-gender-recognition/train.csv")
path = "/kaggle/input/bitmoji-faces-gender-recognition/BitmojiDataset/trainimages/"
path_test = "/kaggle/input/bitmoji-faces-gender-recognition/BitmojiDataset/testimages/"

In [3]:
y = data['is_male']
x = data['image_id']
x_test, x_val_name, y_test, y_val_single = train_test_split(x, y, test_size=0.1, random_state=0)
x_y = np.stack((x_test, y_test), axis=1)
print(x_test.shape, y_test.shape, x_val_name.shape, x_y.shape)
print(x_y)

(2700,) (2700,) (300,) (2700, 2)
[['0820.jpg' -1]
 ['0418.jpg' -1]
 ['0704.jpg' -1]
 ...
 ['1653.jpg' -1]
 ['2607.jpg' -1]
 ['2732.jpg' -1]]


In [4]:
x_val = list()
for i in x_val_name:
    x_tmp = mpimg.imread(path + i).astype(np.float64)
    x_tmp /= 255
    x_val.append(x_tmp)
y_val = list()
for i in y_val_single:
    y_tmp = [1, 0] if i == 1 else [0, 1]
    y_val.append(y_tmp)
x_val = np.array(x_val)
y_val = np.array(y_val)
print(x_val.shape, y_val.shape)

(300, 384, 384, 3) (300, 2)


In [5]:
count_batch = 0
max_batch = 27
def data_generator():
    global count_batch
    while 1:
        if count_batch == max_batch:
            count_batch = 0
            rdm.shuffle(x_y)
        x_batch = list()
        y_batch = list()
        for i in range(100):
            tmp = count_batch*100+i
            x = mpimg.imread(path + x_y[tmp][0]).astype(np.float64)
            x /= 255
            x_batch.append(x)
            y = [1, 0] if x_y[tmp][1] == 1 else [0, 1]
            y_batch.append(y)
        count_batch += 1
        x_batch = np.array(x_batch)
        y_batch = np.array(y_batch)
        yield(x_batch, y_batch)
count_val = 0
max_val = 3
def val_generator():
    global count_val
    while 1:
        if count_val == max_val:
            count_val = 0
        x_ = x_val[100*count_val:100*(count_val + 1) - 1]
        y_ = y_val[100*count_val:100*(count_val + 1) - 1]
        count_val += 1
        yield(x_, y_)

In [6]:
conv_base = DenseNet169(weights="/kaggle/input/tf-keras-pretrained-model-weights/No Top/densenet169_weights_tf_dim_ordering_tf_kernels_notop.h5", include_top=False, input_shape=(384,384,3))
mx = conv_base.output
mx = GlobalAveragePooling2D()(mx)
mx = Dense(1024, activation='relu')(mx)
predictions = Dense(2, activation='softmax')(mx)

# 构建需要训练的完整模型
model = Model(inputs=conv_base.input, outputs=predictions)

2022-12-13 08:02:39.617581: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [7]:
conv_base.trainable = False

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

early_stopping = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=1, mode='auto', restore_best_weights=True)
checkpoint = ModelCheckpoint('/kaggle/working/bitmoji/model_pretrain_{epoch:02d}-{val_loss:.4f}.hdf5', monitor='val_loss', \
                             verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
tensorboard = TensorBoard(log_dir='/kaggle/working/bitmoji/logs/pretrain', histogram_freq=1, batch_size=max_batch, write_graph=False, write_grads=True, \
                          write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='batch')

history_pretrain = model.fit_generator(
    generator=data_generator(),
    steps_per_epoch=max_batch,
    epochs=5,
    max_queue_size=1,
    validation_data=(x_val, y_val),
    callbacks=[early_stopping, checkpoint, tensorboard]
)

2022-12-13 08:02:45.566466: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-12-13 08:02:45.566572: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-12-13 08:02:45.567768: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-12-13 08:02:47.369644: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
 1/27 [>.............................] - ETA: 14:41 - loss: 0.7688 - acc: 0.4900

2022-12-13 08:03:21.296462: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-12-13 08:03:21.297014: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


 2/27 [=>............................] - ETA: 8:21 - loss: 2.6155 - acc: 0.4850 

2022-12-13 08:03:41.382621: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-12-13 08:03:41.412076: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-12-13 08:03:41.445438: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: /kaggle/working/bitmoji/logs/pretrain/train/plugins/profile/2022_12_13_08_03_41

2022-12-13 08:03:41.455047: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to /kaggle/working/bitmoji/logs/pretrain/train/plugins/profile/2022_12_13_08_03_41/9ef2a5fe14fc.trace.json.gz
2022-12-13 08:03:41.518697: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: /kaggle/working/bitmoji/logs/pretrain/train/plugins/profile/2022_12_13_08_03_41

2022-12-13 08:03:41.527342: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to /kaggle/worki

27/27 [==============================] - 628s 23s/step - loss: 0.5541 - acc: 0.8296 - val_loss: 0.2022 - val_acc: 0.9100

Epoch 00001: val_loss improved from inf to 0.20219, saving model to /kaggle/working/bitmoji/model_pretrain_01-0.2022.hdf5
Epoch 2/5
27/27 [==============================] - 608s 23s/step - loss: 0.1970 - acc: 0.9300 - val_loss: 0.1170 - val_acc: 0.9333

Epoch 00002: val_loss improved from 0.20219 to 0.11695, saving model to /kaggle/working/bitmoji/model_pretrain_02-0.1170.hdf5
Epoch 3/5
27/27 [==============================] - 605s 22s/step - loss: 0.1405 - acc: 0.9493 - val_loss: 0.0837 - val_acc: 0.9700

Epoch 00003: val_loss improved from 0.11695 to 0.08372, saving model to /kaggle/working/bitmoji/model_pretrain_03-0.0837.hdf5
Epoch 4/5
27/27 [==============================] - 605s 22s/step - loss: 0.0637 - acc: 0.9726 - val_loss: 0.0780 - val_acc: 0.9667

Epoch 00004: val_loss improved from 0.08372 to 0.07801, saving model to /kaggle/working/bitmoji/model_pretra

In [8]:
for layer in model.layers[:369]:
   layer.trainable = False
for layer in model.layers[369:]:
   layer.trainable = True

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy', metrics=['acc'])

early_stopping = EarlyStopping(monitor='val_loss', min_delta=2e-3, patience=5, verbose=1, mode='auto', restore_best_weights=True)
checkpoint = ModelCheckpoint('/kaggle/working/bitmoji/model_{epoch:02d}-{val_loss:.4f}.hdf5', monitor='val_loss', \
                             verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
tensorboard = TensorBoard(log_dir='/kaggle/working/bitmoji/logs/train', histogram_freq=1, batch_size=max_batch, write_graph=True, write_grads=True, \
                          write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='batch')

history = model.fit_generator(
    generator=data_generator(),
    steps_per_epoch=max_batch,
    epochs=20,
    max_queue_size=1,
    validation_data=(x_val, y_val),
    callbacks=[early_stopping, checkpoint, tensorboard]
)

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2022-12-13 08:54:10.099203: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-12-13 08:54:10.099261: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-12-13 08:54:10.099404: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/20
 1/27 [>.............................] - ETA: 17:48 - loss: 0.0022 - acc: 1.0000

2022-12-13 08:54:52.723228: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-12-13 08:54:52.723421: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


 2/27 [=>............................] - ETA: 11:34 - loss: 0.0020 - acc: 1.0000

2022-12-13 08:55:21.217170: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-12-13 08:55:21.276480: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-12-13 08:55:21.322102: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: /kaggle/working/bitmoji/logs/train/train/plugins/profile/2022_12_13_08_55_21

2022-12-13 08:55:21.336701: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to /kaggle/working/bitmoji/logs/train/train/plugins/profile/2022_12_13_08_55_21/9ef2a5fe14fc.trace.json.gz
2022-12-13 08:55:21.438183: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: /kaggle/working/bitmoji/logs/train/train/plugins/profile/2022_12_13_08_55_21

2022-12-13 08:55:21.449715: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to /kaggle/working/bitmoj

27/27 [==============================] - 784s 29s/step - loss: 0.0151 - acc: 0.9959 - val_loss: 0.3791 - val_acc: 0.8767

Epoch 00001: val_loss improved from inf to 0.37908, saving model to /kaggle/working/bitmoji/model_01-0.3791.hdf5
Epoch 2/20
27/27 [==============================] - 762s 28s/step - loss: 0.0025 - acc: 1.0000 - val_loss: 0.2251 - val_acc: 0.9233

Epoch 00002: val_loss improved from 0.37908 to 0.22514, saving model to /kaggle/working/bitmoji/model_02-0.2251.hdf5
Epoch 3/20
27/27 [==============================] - 758s 28s/step - loss: 0.0018 - acc: 1.0000 - val_loss: 0.1385 - val_acc: 0.9533

Epoch 00003: val_loss improved from 0.22514 to 0.13849, saving model to /kaggle/working/bitmoji/model_03-0.1385.hdf5
Epoch 4/20
27/27 [==============================] - 758s 28s/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.0926 - val_acc: 0.9667

Epoch 00004: val_loss improved from 0.13849 to 0.09258, saving model to /kaggle/working/bitmoji/model_04-0.0926.hdf5
Epoch 5/20
27/2

In [9]:
print("pretrain history:")
print(history_pretrain.epoch)
print(history_pretrain.history)
print("train history:")
print(history.epoch)
print(history.history)

pretrain history:
[0, 1, 2, 3, 4]
{'loss': [0.5540735721588135, 0.1970006376504898, 0.14054395258426666, 0.06369644403457642, 0.00868168007582426], 'acc': [0.8296296000480652, 0.9300000071525574, 0.9492592811584473, 0.9725925922393799, 0.9988889098167419], 'val_loss': [0.20219452679157257, 0.11695271730422974, 0.08372467011213303, 0.07800912857055664, 0.6193128228187561], 'val_acc': [0.9100000262260437, 0.9333333373069763, 0.9700000286102295, 0.9666666388511658, 0.7766666412353516]}
train history:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
{'loss': [0.01514403335750103, 0.0025170971639454365, 0.0018280870281159878, 0.0011194393737241626, 0.00084327143849805, 0.0006261746748350561, 0.0004523951793089509, 0.00036857122904621065, 0.00033238076139241457, 0.0003047329082619399, 0.00028376534464769065], 'acc': [0.9959259033203125, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 'val_loss': [0.37907662987709045, 0.22513872385025024, 0.13849057257175446, 0.09258077293634415, 0.0714050829410553, 0.0

In [10]:
with open("pretrain_history.pkl", 'wb') as fp:
    pkl.dump(history_pretrain.history, fp)
with open("history.pkl", 'wb') as fp:
    pkl.dump(history_pretrain.history, fp)

In [11]:
model.save("/kaggle/working/bitmoji/model" + str(datetime.now()) + ".h5")

In [12]:
test = pd.read_csv("/kaggle/input/bitmoji-faces-gender-recognition/sample_submission.csv")
test_name = test['image_id']

In [13]:
x_test = list()
for name in test_name:
    x_tmp = mpimg.imread(path_test + name).astype(np.float64)
    x_tmp /= 255
    x_test.append(x_tmp)
x_test = np.array(x_test)
print(x_test.shape)

(1084, 384, 384, 3)


In [14]:
y_pred = model.predict(x_test)

In [15]:
print(y_pred)

[[1.4909953e-06 9.9999845e-01]
 [9.3311928e-02 9.0668803e-01]
 [2.0473009e-07 9.9999976e-01]
 ...
 [1.0000000e+00 2.1512512e-09]
 [9.9030853e-12 1.0000000e+00]
 [1.0000000e+00 2.2417620e-08]]


In [16]:
result = pd.DataFrame(columns=['image_id', 'is_male'])
for i in range(1084):
    x = test_name[i]
    y = '1' if y_pred[i][0] > y_pred[i][1] else '-1'
    x_y = {'image_id':x, 'is_male':y}
    result = result.append(x_y, ignore_index=True)
print(result)

      image_id is_male
0     3000.jpg      -1
1     3001.jpg      -1
2     3002.jpg      -1
3     3003.jpg       1
4     3004.jpg      -1
...        ...     ...
1079  4079.jpg      -1
1080  4080.jpg       1
1081  4081.jpg       1
1082  4082.jpg      -1
1083  4083.jpg       1

[1084 rows x 2 columns]


In [17]:
#file_name = 'result_' + str(datetime.now()).replace(' ', '_') + '.csv'
file_name='submission.csv'
'''with open(file_name, encoding = 'utf8') as fp:
    result.to_csv(file_name, index = False)'''
result.to_csv(file_name, index = False)